In [2]:
# libs
import re
import pandas as pd 
import numpy as np 
from datetime import datetime

In [70]:
education = pd.read_csv('../dataset/education.csv')
language = pd.read_csv('../dataset/languages.csv')
skills = pd.read_csv('../dataset/skills.csv')
experience = pd.read_csv('../dataset/work_experiences.csv')
"datasets loaded!"

'datasets loaded!'

<hr>

### Education

In [3]:
education.head()

,user_id,school_name,degree,fields_of_study,start_year_month,end_year_month
0,0,Anadolu Üniversitesi,NaN,NaN,NaN,NaN
1,0,Adıyaman Üniversitesi,NaN,NaN,NaN,NaN
2,0,Fırat Üniversitesi,NaN,NaN,NaN,NaN
3,1,Fırat Üniversitesi,Yüksek Lisans,Yazılım Mühendisliği,NaN,NaN
4,1,Fırat Üniversitesi,Lisans,Yazılım Mühendisliği,NaN,NaN


In [4]:
education.isnull().sum() / len(education) * 100

user_id              0.000000
school_name          0.000000
degree              25.139050
fields_of_study     13.914080
start_year_month    94.247238
end_year_month      94.676486
dtype: float64

In [5]:
education = education[['user_id', 'school_name', 'fields_of_study']]
education.head()

,user_id,school_name,fields_of_study
0,0,Anadolu Üniversitesi,NaN
1,0,Adıyaman Üniversitesi,NaN
2,0,Fırat Üniversitesi,NaN
3,1,Fırat Üniversitesi,Yazılım Mühendisliği
4,1,Fırat Üniversitesi,Yazılım Mühendisliği


In [6]:
education['fields_of_study'].value_counts()

Bilgisayar Mühendisliği                      14698
Computer Engineering                         12662
Elektrik ve Elektronik Mühendisliği           3562
Electrical and Electronics Engineering        3201
Computer Science                              3114
                                             ...  
Siber Güvenlik Eğitimi                           1
Bilgi Sistemleri Temel Sertifika Programı        1
Ceng                                             1
Big data                                         1
Trade and Tourism                                1
Name: fields_of_study, Length: 11048, dtype: int64

In [7]:
mapping_fields_of_study = {
    "computer engineering": [
        "bilgisayar mühendisliği",
        "computer engineering",
        "bilgisayar yazılımı mühendisliği",
        "computer software engineering",
        "computer science and engineering",
        "computer science",
        "computer engineer"
    ],
    "electrical and electronics engineering": [
        "elektrik ve elektronik mühendisliği",
        "electrical and electronics engineering",
        "elektronik ve haberleşme mühendisliği",
        "electronics engineering",
        "electronics and communication engineering",
        "electronics and communications engineering",
        "electrical, electronics and communications engineering",
        "electrical engineering"
    ],
    "business administration and management": [
        "yönetim bilişim sistemleri",
        "i̇şletme",
        "i̇şletme ve yönetim, genel",
        "business administration and management, general",
        "business administration",
        "mba",
        "yönetim bilgi sistemleri, genel",
        "management information systems",
        "management information systems, general"
    ],
    "mechanical engineering": [
        "makine mühendisliği",
        "mechanical engineering"
    ],
    "industrial engineering": [
        "endüstri mühendisliği",
        "industrial engineering"
    ],
    "mathematics": [
        "bilgisayar programlama",
        "mathematics",
        "matematik",
        "science",
        "fen bilimleri",
        "matematik mühendisliği"
    ],
    "computer programming": [
        "computer programming",
        "bilgisayar programlama/programcı, genel",
        "bilgisayar programcılığı"
    ],
    "software engineering": [
        "yazılım mühendisliği",
        "software engineering"
    ],
    "information technology": [
        "information technology"
    ],
    "economics": [
        "economics"
    ],
    "physics": [
        "physics",
        "fizik"
    ],
    "chemical engineering": [
        "kimya mühendisliği",
        "chemical engineering"
    ],
    "engineering management": [
        "engineering management"
    ],
    "civil engineering": [
        "civil engineering"
    ],
    "mechatronics, robotics and automation engineering": [
        "mekatronik, robotik ve otomasyon mühendisliği"
    ]
}

In [8]:
education['school_name'] = education['school_name'].str.lower()
education['school_name'].value_counts()[:15]

anadolu üniversitesi             6568
i̇stanbul üniversitesi           4891
sakarya üniversitesi             4311
kocaeli üniversitesi             3844
i̇stanbul teknik üniversitesi    3835
istanbul technical university    3793
yıldız teknik üniversitesi       3776
marmara üniversitesi             3573
hacettepe üniversitesi           2992
ege üniversitesi                 2846
gazi üniversitesi                2806
bahçeşehir üniversitesi          2500
istanbul university              2317
dokuz eylül üniversitesi         2303
yildiz technical university      2114
Name: school_name, dtype: int64

In [9]:
school_name_map = {
    "anadolu üniversitesi":['anadolu üniversitesi', 'anadolu university'],
    "istanbul üniversitesi":["istanbul üniversitesi", "istanbul university"],
    "sakarya üniversitesi":["sakarya üniversitesi", "sakarya university"],
    "kocaeli üniversitesi":["kocaeli üniversitesi", "kocaeli university"],
    "ITU":["istanbul teknik üniversitesi", "istanbul technical university"],
    "YTU":["yıldız teknik üniversitesi", "yıldız technical university"],
    "marmara üniversitesi":["marmara üniversitesi", "marmara university"],
    "hacettepe üniversitesi":["hacettepe üniversitesi", "hacettepe university"],
    "ege üniversitesi":["ege üniversitesi", "ege university"]
}

In [11]:
def mapping(value, map):
    if isinstance(value, float) and pd.isna(value):
        return value
    value = re.sub(r'[\.,;]', '', value).lower()
    for mapped, keywords in map.items():
        for keyword in keywords:
            keyword = keyword.lower()
            if re.search(r"\b" + keyword + r"\b", value):
                return mapped
    return "diğer"

In [11]:
education['fields_of_study'] = education['fields_of_study'].apply(lambda x: mapping(x, mapping_fields_of_study))
education['fields_of_study'].value_counts()

diğer                                     45288
computer engineering                      34693
electrical and electronics engineering    10676
mathematics                                9437
business administration and management     9247
computer programming                       2694
mechanical engineering                     2340
industrial engineering                     2265
software engineering                       1643
physics                                    1295
chemical engineering                        881
economics                                   833
information technology                      721
engineering management                      383
civil engineering                           341
Name: fields_of_study, dtype: int64

In [12]:
education['school_name'] = education['school_name'].apply(lambda x: mapping(x, school_name_map))
education['school_name'].value_counts()

diğer                     100155
anadolu üniversitesi        8052
sakarya üniversitesi        5306
marmara üniversitesi        5034
kocaeli üniversitesi        4869
YTU                         4608
hacettepe üniversitesi      4130
ITU                         4047
ege üniversitesi            3870
istanbul üniversitesi       2504
Name: school_name, dtype: int64

In [13]:
education.head()

,user_id,school_name,fields_of_study
0,0,anadolu üniversitesi,NaN
1,0,diğer,NaN
2,0,diğer,NaN
3,1,diğer,software engineering
4,1,diğer,software engineering


In [14]:
education['fields_of_study'] = education['fields_of_study'].fillna("diğer")
education.isnull().sum()

user_id            0
school_name        0
fields_of_study    0
dtype: int64

In [15]:
education.head()

,user_id,school_name,fields_of_study
0,0,anadolu üniversitesi,diğer
1,0,diğer,diğer
2,0,diğer,diğer
3,1,diğer,software engineering
4,1,diğer,software engineering


In [16]:
education['school_name'] = education['school_name'].apply(lambda x: x.replace(' ', '_'))
education['fields_of_study'] = education['fields_of_study'].apply(lambda x: x.replace(' ', '_'))
education.head()

,user_id,school_name,fields_of_study
0,0,anadolu_üniversitesi,diğer
1,0,diğer,diğer
2,0,diğer,diğer
3,1,diğer,software_engineering
4,1,diğer,software_engineering


In [17]:
edu_ohe1 = education[['user_id']].join(pd.get_dummies(education['school_name']).add_prefix('')).groupby('user_id').max().reset_index()
edu_ohe2 = education[['user_id']].join(pd.get_dummies(education['fields_of_study']).add_prefix('')).groupby('user_id').max().reset_index()
edu_ohe = pd.merge(edu_ohe1, edu_ohe2, on='user_id', how='left')
edu_ohe.head()

,user_id,ITU,YTU,anadolu_üniversitesi,diğer_x,ege_üniversitesi,hacettepe_üniversitesi,istanbul_üniversitesi,kocaeli_üniversitesi,marmara_üniversitesi,...,diğer_y,economics,electrical_and_electronics_engineering,engineering_management,industrial_engineering,information_technology,mathematics,mechanical_engineering,physics,software_engineering
0,0,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
grouped = education.groupby("user_id")
school_count = grouped.size().reset_index(name="school_count")
school_count.head()

,user_id,school_count
0,0,3
1,1,2
2,2,2
3,3,1
4,4,1


In [19]:
education_new = pd.merge(edu_ohe, school_count, on='user_id', how='left')
education_new.head()

,user_id,ITU,YTU,anadolu_üniversitesi,diğer_x,ege_üniversitesi,hacettepe_üniversitesi,istanbul_üniversitesi,kocaeli_üniversitesi,marmara_üniversitesi,...,economics,electrical_and_electronics_engineering,engineering_management,industrial_engineering,information_technology,mathematics,mechanical_engineering,physics,software_engineering,school_count
0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
2,2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [20]:
education_new.columns = ['user_id', 'ITU', 'YTU', 'anadolu_üniversitesi', 'diğer_school',
       'ege_üniversitesi', 'hacettepe_üniversitesi', 'istanbul_üniversitesi',
       'kocaeli_üniversitesi', 'marmara_üniversitesi', 'sakarya_üniversitesi',
       'business_administration_and_management', 'chemical_engineering',
       'civil_engineering', 'computer_engineering', 'computer_programming',
       'diğer_fields_of_study', 'economics', 'electrical_and_electronics_engineering',
       'engineering_management', 'industrial_engineering',
       'information_technology', 'mathematics', 'mechanical_engineering',
       'physics', 'software_engineering', 'school_count']
education_new.head()

,user_id,ITU,YTU,anadolu_üniversitesi,diğer_school,ege_üniversitesi,hacettepe_üniversitesi,istanbul_üniversitesi,kocaeli_üniversitesi,marmara_üniversitesi,...,economics,electrical_and_electronics_engineering,engineering_management,industrial_engineering,information_technology,mathematics,mechanical_engineering,physics,software_engineering,school_count
0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
2,2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [21]:
education_new.to_csv('../Cleaned Datasets/education.csv', index=False)

<hr>

### Language

In [22]:
language.head()

,user_id,language,proficiency
0,8,İngilizce,full_professional
1,8,Türkçe,native_or_bilingual
2,8,Fransızca,elementary
3,10,ingilizce,NaN
4,11,Turkish,native_or_bilingual


In [23]:
language['language'].value_counts()[:20] / len(language) * 100

İngilizce                      27.217270
English                        19.540638
Türkçe                         18.415240
Turkish                        10.860877
Almanca                         4.906524
German                          4.072993
French                          1.227946
Spanish                         1.104362
İspanyolca                      1.100418
Fransızca                       0.995241
ingilizce                       0.767795
Rusça                           0.706003
Russian                         0.683653
Arabic                          0.682338
Arapça                          0.506166
Italian                         0.461466
İtalyanca                       0.437801
İngilizce, Orta (1100-1500)     0.433857
Japanese                        0.362862
Deutsch                         0.266888
Name: language, dtype: float64

In [24]:
language_map = {
    "ingilizce": ["ingilzice","english","i̇ngilizce, eski (yaklaşık 450-1100)","i̇ngilizce, orta (1100-1500)"], 
    "türkçe":['türkçe','turkish','türkçe, osmanlıca (1500-1928)'],
    "almanca":['germany','german','almanca','deutch','deutsch'],
    "fransızca":['french','fransızca'],
    "ispanyolca":['spanish','ispanyolca'],
    "rusça":['russian','rusça'],
    "arapça":['arapça','arabian','arabic'],
    "italyanca":['italian','italyanca'],
    "japonca":['japanese','japonca'],
    "azerice":['azerbaijani']
}

In [25]:
language['language'] = language['language'].str.lower()
language['proficiency'] = language['proficiency'].str.lower()
language['language'] = language['language'].apply(lambda x: mapping(x, language_map))
language['language'].value_counts()

diğer         25809
türkçe        22512
ingilizce     15008
almanca        7184
fransızca      1709
rusça          1066
arapça          935
ispanyolca      865
japonca         442
italyanca       380
azerice         152
Name: language, dtype: int64

In [26]:
language.head()

,user_id,language,proficiency
0,8,diğer,full_professional
1,8,türkçe,native_or_bilingual
2,8,fransızca,elementary
3,10,diğer,NaN
4,11,türkçe,native_or_bilingual


In [27]:
language.isnull().sum() / len(language) * 100

user_id         0.000000
language        0.000000
proficiency    14.016197
dtype: float64

In [28]:
language['proficiency'].value_counts()

native_or_bilingual     22026
professional_working    18306
elementary               8971
full_professional        8389
limited_working          7709
Name: proficiency, dtype: int64

In [29]:
language['proficiency'] = language['proficiency'].fillna('limited_working')
language.isnull().sum()

user_id        0
language       0
proficiency    0
dtype: int64

In [30]:
language.sort_values(by='user_id').head()

,user_id,language,proficiency
0,8,diğer,full_professional
1,8,türkçe,native_or_bilingual
2,8,fransızca,elementary
3,10,diğer,limited_working
4,11,türkçe,native_or_bilingual


In [31]:
lang_ohe1 = language[['user_id']].join(pd.get_dummies(language['language'])).groupby('user_id').max().reset_index()
lang_ohe2 = language[['user_id']].join(pd.get_dummies(language['proficiency'])).groupby('user_id').max().reset_index()
lang_ohe = pd.merge(lang_ohe1, lang_ohe2, on='user_id', how='left')
lang_ohe.head()

,user_id,almanca,arapça,azerice,diğer,fransızca,ingilizce,ispanyolca,italyanca,japonca,rusça,türkçe,elementary,full_professional,limited_working,native_or_bilingual,professional_working
0,8,0,0,0,1,1,0,0,0,0,0,1,1,1,0,1,0
1,10,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,11,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1
3,12,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1
4,13,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0


In [32]:
grouped_language = language.groupby('user_id')
language_count = grouped_language.size().reset_index(name="language_count")
language_count.head()

,user_id,language_count
0,8,3
1,10,1
2,11,3
3,12,2
4,13,2


In [33]:
language_new = pd.merge(lang_ohe, language_count, on='user_id', how='left')
language_new.head()

,user_id,almanca,arapça,azerice,diğer,fransızca,ingilizce,ispanyolca,italyanca,japonca,rusça,türkçe,elementary,full_professional,limited_working,native_or_bilingual,professional_working,language_count
0,8,0,0,0,1,1,0,0,0,0,0,1,1,1,0,1,0,3
1,10,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1
2,11,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,3
3,12,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,2
4,13,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,2


In [34]:
language_new.to_csv('../Cleaned Datasets/languages.csv', index=False)

<hr>

### Skills

In [35]:
skills.head()

,user_id,skill
0,1,Mühendislik
1,1,Eğitim
2,2,Android
3,2,Java
4,2,3D Studio Max


In [36]:
len(skills['skill'].value_counts()) / len(skills), len(skills['skill'].value_counts())

(0.03791716930901009, 53025)

In [37]:
skills['skill'].value_counts()

Java                         24360
JavaScript                   24072
SQL                          23590
C#                           23422
HTML                         16872
                             ...  
CSWA                             1
CompTIA A+                       1
Virtual Machine                  1
Microsoft Sistem Yönetimi        1
Patient Relations                1
Name: skill, Length: 53025, dtype: int64

In [38]:
map_skills = {
'Java': 'Java',
'JavaScript': 'JavaScript',
'SQL': 'SQL',
'C#': 'C#',
'HTML': 'HTML',
'CSS': 'CSS',
'C++': 'C++',
'Microsoft SQL Server': 'SQL',
'C': 'C',
'Microsoft Office': 'Microsoft Office',
'Python': 'Python',
'MySQL': 'SQL',
'Git': 'Git',
'Linux': 'Linux',
'Software Development': 'Software Development',
'jQuery': 'jQuery',
'ASP.NET': 'ASP.NET',
'HTML5': 'HTML',
'.NET': '.NET',
'ASP.NET MVC': 'ASP.NET',
'PHP': 'PHP',
'Yazılım Geliştirme': 'Software Development',
'OOP (Nesne Yönelimli Programlama)': 'OOP',
'Visual Studio': 'Visual Studio',
'Matlab': 'Matlab',
'XML': 'XML',
'React.js': 'React.js',
'Microsoft Excel': 'Microsoft Excel',
'İngilizce': 'English',
'Scrum': 'Scrum',
'JSON': 'JSON',
'Android': 'Android',
'PL/SQL': 'SQL',
'Web Services': 'Web Development',
'Project Management': 'Project Management',
'Spring Framework': 'Spring Framework',
'JIRA': 'JIRA',
'PostgreSQL': 'SQL',
'Proje Yönetimi': 'Project Management',
'Software Engineering': 'Software Engineering',
'Web Geliştirme': 'Web Development',
'OOP': 'OOP',
'Hibernate': 'Hibernate',
'Web Development': 'Web Development',
'Object-Oriented Programming (OOP)': 'OOP',
'Eclipse': 'Eclipse',
'AJAX': 'AJAX',
'Node.js': 'Node.js',
'T-SQL': 'SQL',
'Photoshop': 'Photoshop',
'Ekip Çalışması': 'Teamwork',
'English': 'English',
'Docker': 'Docker',
'MongoDB': 'MongoDB',
'Mühendislik': 'Engineering',
'Agile Methodologies': 'Agile Methodologies',
'Microsoft Word': 'Microsoft Word',
'AutoCAD': 'AutoCAD',
'Entity Framework': 'Entity Framework',
'Programming': 'Programming',
}

In [39]:
skills['skill'] = skills['skill'].map(map_skills)
skills['skill'].replace(np.nan, 'other', inplace=True)
skills['skill'].replace('', 'other', inplace=True)
skills['skill'].value_counts()

other                   903299
SQL                      65683
HTML                     26252
Java                     24360
JavaScript               24072
C#                       23422
Software Development     19221
ASP.NET                  18984
OOP                      16650
CSS                      15109
C++                      14762
C                        14121
Web Development          13992
Microsoft Office         13854
Python                   13353
Git                      12651
Linux                    11552
jQuery                   10497
English                  10089
Project Management        9884
.NET                      8963
PHP                       8444
Visual Studio             7694
Matlab                    7485
XML                       6901
React.js                  6498
Microsoft Excel           6433
Scrum                     5901
JSON                      5787
Android                   5687
Spring Framework          5170
JIRA                      4898
Software

In [40]:
skills.isnull().sum()

user_id    0
skill      0
dtype: int64

In [41]:
skills.head()

,user_id,skill
0,1,Engineering
1,1,other
2,2,Android
3,2,Java
4,2,other


In [46]:
skills_count = skills.groupby('user_id').count()
skills_count.columns = ['skill_count']
skills_count = skills_count.reset_index()
skills_count.head()

,user_id,skill_count
0,1,2
1,2,6
2,3,3
3,5,11
4,6,6


In [47]:
skills_ohe = skills[['user_id']].join(pd.get_dummies(skills['skill'])).groupby('user_id').max().reset_index()
skills_ohe.head()

,user_id,.NET,AJAX,ASP.NET,Agile Methodologies,Android,AutoCAD,C,C#,C++,...,Scrum,Software Development,Software Engineering,Spring Framework,Teamwork,Visual Studio,Web Development,XML,jQuery,other
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [48]:
skills_new = pd.merge(skills_ohe, skills_count, on='user_id', how='left')
skills_new.head()

,user_id,.NET,AJAX,ASP.NET,Agile Methodologies,Android,AutoCAD,C,C#,C++,...,Software Development,Software Engineering,Spring Framework,Teamwork,Visual Studio,Web Development,XML,jQuery,other,skill_count
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
1,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,11
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6


In [49]:
skills_new.to_csv('../Cleaned Datasets/skills.csv', index=False)

<hr>

### Experience

In [71]:
experience.head()

,user_id,company_id,location,start_year_month
0,53442,2651,"Istanbul, Turkey",201509
1,34558,815,"Istanbul, Turkey",201210
2,63761,26354,NaN,200010
3,10738,89,NaN,201610
4,8711,3113,"Istanbul, Turkey",201801


In [13]:
location_map={
    "istanbul":["istanbul", "istanbul, turkey", "ümraniye",
                "maslak","ümraniye, istanbul, türkiye", "beşiktaş",
                "ataşehir, istanbul, türkiye","istanbul, istanbul, turkey",
                "kadikoy, istanbul, turkey", "pendik, istanbul, turkey",
                "maltepe, istanbul, turkey", "üsküdar, istanbul, turkey",
                "beşiktaş, istanbul, turkey", "besiktas, istanbul, turkey",
                "sariyer, istanbul, turkey", "atasehir, istanbul, turkey",
                "kartal, istanbul, turkey", "sisli, istanbul, turkey",
                "umraniye, istanbul, turkey", "kagithane, istanbul, turkey",
                "bakirkoy, istanbul, turkey", "tuzla, istanbul, turkey",
                "bahcelievler, istanbul, turkey", "beylikduzu, istanbul, turkey",
                "avcilar, istanbul, turkey", "kucukcekmece, istanbul, turkey",
                "greater istanbul", "sancaktepe, istanbul, turkey",
                "beykoz, istanbul, turkey", "fatih, istanbul, turkey",
                "zeytinburnu, istanbul, turkey"],
    "ankara":["ankara", "çankaya, ankara, türkiye",
              "ankara, ankara, turkey", "ankara, turkey",
              "cankaya, ankara, turkey", "yenimahalle, ankara, turkey",
              "altindag, ankara, turkey"],
    "izmir":["izmir", "izmir, türkiye", "izmir, turkey",
             "bornova, izmir, turkey", "izmir, izmir, turkey"],
    "kocaeli":["kocaeli", "gebze, kocaeli", "kocaeli, turkey",
               "izmit, kocaeli, turkey", "gebze, kocaeli, turkey"],
    "bursa":["bursa", "bursa, turkey"],
    "eskisehir":["eskisehir", "eskişehir", "eskisehir, turkey"],
    "sakarya":["sakarya", "sakarya, turkey", "serdivan, sakarya, turkey"],
    "antalya":["antalya", "antalya, turkey"],
    "manisa":["manisa", "manisa, turkey"],
    "kayseri":["kayseri", "kayseri, turkey"],
    "adana":["adana", "adana, turkey"],
    "konya":["konya", "konya, turkey"],
    "trabzon":["trabzon", "trabzon, turkey"],
    "mersin":["mersin", "mersin, turkey", "mercin, içel, turkey"],
    "tekirdağ":["tekirdağ", "tekirdağ, türkiye"],
    "mugla":["mugla", "mugla, turkey", "muğla, turkey"],
    "denizli":["denizli", "denizli, turkey"],
    "samsun":["samsun", "samsun, turkey"],
    "elazig":["elazig", "elazığ, türkiye", "elazig, turkey"],
    "US":["us"],
    "isparta":["isparta, turkey"],
    "gaziantep":["gaziantep, turkey"],
    "edirne":["edirne, turkey"],
    "sivas":["sivas, turkey"],
    "kahramanmaras":["kahramanmaras, turkey"]
}

In [73]:
experience['location'] = experience['location'].str.lower()
experience['location'] = experience['location'].apply(lambda x: mapping(x, location_map))
experience['location'].value_counts()

istanbul     47725
diğer        38158
ankara       20810
izmir         4454
kocaeli       4417
bursa         1525
eskisehir     1504
sakarya       1195
antalya       1037
manisa        1029
kayseri        744
adana          731
konya          687
trabzon        368
mersin         310
tekirdağ       198
denizli        186
mugla          183
samsun         178
elazig         149
US              17
Name: location, dtype: int64

In [74]:
experience['start_year_month'] = pd.to_datetime(experience['start_year_month'], format='%Y%m')
experience.head()

,user_id,company_id,location,start_year_month
0,53442,2651,istanbul,2015-09-01
1,34558,815,istanbul,2012-10-01
2,63761,26354,NaN,2000-10-01
3,10738,89,NaN,2016-10-01
4,8711,3113,istanbul,2018-01-01


In [75]:
experience.isnull().sum() / len(experience) * 100

user_id              0.000000
company_id           0.000000
location            32.838375
start_year_month     0.000000
dtype: float64

In [76]:
experience['location'] = experience['location'].fillna('diğer')
experience.isnull().sum(),"*"*50, experience.head()

(user_id             0
 company_id          0
 location            0
 start_year_month    0
 dtype: int64,
 '**************************************************',
    user_id  company_id  location start_year_month
 0    53442        2651  istanbul       2015-09-01
 1    34558         815  istanbul       2012-10-01
 2    63761       26354     diğer       2000-10-01
 3    10738          89     diğer       2016-10-01
 4     8711        3113  istanbul       2018-01-01)

In [77]:
experience.drop_duplicates().sort_values(by=['user_id','start_year_month']).head(10)

,user_id,company_id,location,start_year_month
147720,0,0,diğer,2005-09-01
760,2,9,elazig,2016-12-01
19762,2,7,elazig,2017-06-01
180157,2,10,mersin,2018-06-01
30839,5,15,diğer,2017-06-01
161935,7,21,istanbul,2016-07-01
171458,7,21,istanbul,2017-07-01
54420,7,21,diğer,2017-08-01
17649,7,20,diğer,2018-12-01
4573,10,27,elazig,2013-08-01


In [80]:
now = datetime.now()

exp_date = experience.groupby('user_id')['start_year_month'].min().reset_index()
exp_date['month_difference'] = (now.year - exp_date['start_year_month'].dt.year) * 12 + (now.month - exp_date['start_year_month'].dt.month) - 1
exp_date = exp_date[['user_id', 'month_difference']]
exp_date.columns = ['user_id', 'experience_month']
exp_date.head()

,user_id,experience_month
0,0,208
1,2,73
2,5,67
3,7,78
4,10,113


In [83]:
exp_grouped = experience.drop_duplicates().sort_values(by='user_id').groupby('user_id')
exp_count = exp_grouped.size().reset_index(name="exp_comp_count")
exp_count.head()

,user_id,exp_comp_count
0,0,1
1,2,3
2,5,1
3,7,4
4,10,2


In [82]:
exp_ohe = experience[['user_id']].join(pd.get_dummies(experience['location'])).groupby('user_id').max().reset_index()
exp_ohe.head()

,user_id,US,adana,ankara,antalya,bursa,denizli,diğer,elazig,eskisehir,...,kayseri,kocaeli,konya,manisa,mersin,mugla,sakarya,samsun,tekirdağ,trabzon
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,5,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
experience_new = pd.merge(exp_ohe, exp_count, on='user_id', how='left')
experience_new = pd.merge(experience_new, exp_date, on='user_id', how='left')
experience_new.head()

,user_id,US,adana,ankara,antalya,bursa,denizli,diğer,elazig,eskisehir,...,konya,manisa,mersin,mugla,sakarya,samsun,tekirdağ,trabzon,exp_comp_count,experience_month
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,208
1,2,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,3,73
2,5,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,67
3,7,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,4,78
4,10,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,2,113


In [86]:
experience_new.to_csv('../Cleaned Datasets/experience.csv', index=False)

<hr>

### Train

In [3]:
train = pd.read_csv('../dataset/train_users.csv')
train.head()

,user_id,industry,location,moved_after_2019
0,1301,Information Technology and Services,"Istanbul, Istanbul, Turkey",1
1,6950,Internet,"Istanbul, Istanbul, Turkey",0
2,4880,Online Media,Turkey,0
3,26046,Telecommunications,"Istanbul, Istanbul, Turkey",0
4,11005,Banking,"Istanbul, Turkey",0


<hr>

### Test

In [4]:
test = pd.read_csv('../dataset/test_users.csv')
test.head()

,user_id,industry,location
0,17449,Research,Turkey
1,33967,Computer Software,"Istanbul, Istanbul, Turkey"
2,2110,Automotive,Turkey
3,55082,Internet,Turkey
4,37165,Electrical/Electronic Manufacturing,Turkey


<hr>

### Train & Test Industry

In [5]:
dict1 = train['industry'].value_counts().to_dict()
dict2 = test['industry'].value_counts().to_dict()

for key in dict2:
    if key in dict1:
        dict1[key] += dict2[key]
    else:
        dict1[key] = dict2[key]

industry = pd.DataFrame.from_dict(dict1, orient='index', columns=['value'])
industry.head()

,value
Computer Software,21457
Information Technology and Services,13816
Telecommunications,3647
Defense & Space,2880
Banking,2016


In [6]:
more_q3 = industry[industry['value'] > industry['value'].describe()['75%']].index.tolist()
more_q3[:5]

['Computer Software',
 'Information Technology and Services',
 'Telecommunications',
 'Defense & Space',
 'Banking']

In [7]:
industry_map = {
    "Computer Software": "Information Technology",
    "Information Technology and Services": "Information Technology",
    "Telecommunications": "Information Technology",
    "Defense & Space": "Aerospace and Defense",
    "Banking": "Financial Services",
    "Internet": "Information Technology",
    "Automotive": "Automotive and Manufacturing",
    "Electrical/Electronic Manufacturing": "Automotive and Manufacturing",
    "Computer & Network Security": "Information Technology",
    "Aviation & Aerospace": "Aerospace and Defense",
    "Higher Education": "Education",
    "Financial Services": "Financial Services",
    "Computer Games": "Entertainment and Gaming",
    "Program Development": "Information Technology",
    "Industrial Automation": "Automotive and Manufacturing",
    "Research": "Research and Development",
    "Retail": "Retail and Wholesale",
    "Insurance": "Financial Services",
    "Airlines/Aviation": "Aerospace and Defense",
    "Pharmaceuticals": "Healthcare and Pharmaceuticals",
    "Mechanical or Industrial Engineering": "Automotive and Manufacturing",
    "Consumer Electronics": "Consumer Goods",
    "Nonprofit Organization Management": "Non-Profit",
    "Logistics and Supply Chain": "Logistics and Supply Chain",
    "Oil & Energy": "Energy",
    "Consumer Goods": "Consumer Goods",
    "Education Management": "Education",
    "Food Production": "Food and Beverage",
    "Computer Networking": "Information Technology",
    "Chemicals": "Chemicals and Materials",
    "Marketing and Advertising": "Marketing and Advertising",
    "Textiles": "Textiles and Apparel",
    "Machinery": "Machinery and Equipment",
    "E-Learning": "Education",
    "Construction": "Construction and Engineering"
}

In [8]:
train['industry'] = train['industry'].map(industry_map)
train['industry'].replace(np.nan, 'Others', inplace=True)
train['industry'].replace('', 'Others', inplace=True)
train['industry'].value_counts()

Information Technology            34173
Others                             4323
Aerospace and Defense              3506
Automotive and Manufacturing       2951
Financial Services                 2577
Education                          1076
Consumer Goods                      584
Entertainment and Gaming            567
Research and Development            387
Retail and Wholesale                383
Healthcare and Pharmaceuticals      344
Non-Profit                          305
Logistics and Supply Chain          289
Energy                              286
Food and Beverage                   242
Chemicals and Materials             232
Marketing and Advertising           211
Textiles and Apparel                204
Machinery and Equipment             194
Construction and Engineering        185
Name: industry, dtype: int64

In [14]:
test['industry'] = test['industry'].map(industry_map)
test['industry'].replace(np.nan, 'Others', inplace=True)
test['industry'].replace('', 'Others', inplace=True)
test['industry'].value_counts()

Others                        12360
Financial Services              627
Consumer Goods                  135
Logistics and Supply Chain       77
Marketing and Advertising        56
Name: industry, dtype: int64

<hr>

### Train & Test Location

In [15]:
train['location'] = train['location'].apply(lambda x: mapping(x, location_map))
train['location'].value_counts()

istanbul     28391
diğer        13834
ankara        7708
kocaeli        557
antalya        387
bursa          292
eskisehir      291
adana          288
sakarya        286
kayseri        269
konya          250
manisa         170
samsun          76
tekirdağ        62
denizli         62
mersin          42
trabzon         35
izmir           15
elazig           3
Name: location, dtype: int64

In [16]:
test['location'] = test['location'].apply(lambda x: mapping(x, location_map))
test['location'].value_counts()

istanbul     7081
diğer        3453
ankara       1948
kocaeli       134
antalya       106
kayseri        89
bursa          82
adana          68
sakarya        65
konya          55
eskisehir      49
manisa         44
samsun         23
tekirdağ       18
denizli        15
mersin         12
trabzon        11
izmir           2
Name: location, dtype: int64

In [111]:
train.to_csv('../Cleaned Datasets/train.csv', index=False)
test.to_csv('../Cleaned Datasets/test.csv', index=False)

<hr>